# USAXS plots

code for reproducing Figs. 2,7,S2,S3,S4,S5 and S6

In [ ]:
%matplotlib notebook
%pylab
# %matplotlib inline
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format='retina'

from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle as pkl
from GetConfiguration import get_methods

import pandas as pd
import h5py

from cmcrameri import cm

import matplotlib.patheffects as pe

from Xana import Xana

from matplotlib.colors import ListedColormap

def folder_structure(name, node):
    ''' display content of h5 file '''
    if isinstance(node, h5py.Group):
        print(node.name)
    elif isinstance(node, h5py.Dataset):
        if (node.dtype == 'object') :
            print (node.name, 'contains numeric data')
        else:
            print('\t', node.name, node.shape)
            
def log_ave(x_arr,y_arr,nbins):
    ''' perform logarithmic binning of data

    Parameters
    ----------
    x_arr : array
        array with x values
    y_arr : array
        array with y values
    nbins: int
        number of bins 

    Returns
    -------
    new_x: array
        logarithmically binned x values   
    new_y: array
        logarithmically binned y values
    
    '''
    log_inds = np.insert(np.unique(np.logspace(np.log10(1), np.log10(len(x_arr) - 1), nbins, dtype=int)), 0, 0)
    new_y = np.zeros(np.shape(log_inds))

    
    for ind, log_ind in enumerate(log_inds):
        if ind == 0:
            new_y[ind] = y_arr[0]
        elif log_inds[ind] - log_inds[ind - 1] == 1:
            if log_inds[ind] == ind:
                new_y[ind] = y_arr[ind]
            else:
                new_y[ind] = y_arr[log_inds[ind - 1]:log_inds[ind]]
        elif log_inds[ind] - log_inds[ind - 1] > 1:
            new_y[ind] = np.mean(y_arr[log_inds[ind - 1]:log_inds[ind]])

    new_x = x_arr[log_inds]
    
    return new_x, new_y

def weighted_average(vals,errs):
    ''' calculate weighted average

    Parameters
    ----------
    vals : array
        data points
    errs : array
        errors of data points

    Returns
    -------
    mean: double
        weighted average   
    err: double
        error of weighted average
    
    '''
    eps = 10E-14
    weights = np.divide(1,np.power(errs+eps,2))
    mean = np.sum(np.multiply(weights,vals))/np.sum(weights)
    err = 1/np.sqrt(np.sum(weights))
    return mean, err

def log_ave_errors(x_arr,y_arr,dy_arr,nbins):
    ''' perform logarithmic averaging with errors in y direction

    Parameters
    ----------
    x_arr : array
        array with x values
    y_arr : array
        array with y values
    dy_arr : array
        array with y errors
    nbins: int
        number of bins 

    Returns
    -------
    new_x: array
        logarithmically binned x values   
    new_y: array
        logarithmically binned y values
    new_dy: array
        errors of logarithmically binned y values
    
    '''
    log_inds = np.insert(np.unique(np.logspace(np.log10(1), np.log10(len(x_arr) - 1), nbins, dtype=int)), 0, 0)
    new_y = np.zeros(np.shape(log_inds))
    new_dy = np.zeros(np.shape(log_inds))

    
    for ind, log_ind in enumerate(log_inds):
        if ind == 0:
            new_y[ind] = y_arr[0]
            new_dy[ind] = dy_arr[0]
        elif log_inds[ind] - log_inds[ind - 1] == 1:
            if log_inds[ind] == ind:
                new_y[ind] = y_arr[ind]
                new_dy[ind] = dy_arr[ind]
            else:
                new_y[ind] = y_arr[log_inds[ind - 1]:log_inds[ind]]
                new_dy[ind] = dy_arr[log_inds[ind - 1]:log_inds[ind]]
        elif log_inds[ind] - log_inds[ind - 1] > 1:
            new_y[ind],new_dy[ind] = weighted_average(y_arr[log_inds[ind - 1]:log_inds[ind]],
                                                      dy_arr[log_inds[ind - 1]:log_inds[ind]])
            #new_dy[ind] = np.std(y_arr[log_inds[ind - 1]:log_inds[ind]])/np.sqrt(int(log_inds[ind]-log_inds[ind-1]))

    new_x = x_arr[log_inds]
    
    return new_x, new_y, new_dy

from scipy.constants import Boltzmann,pi,e
def Dosis(t, ab):
    '''Function to calculate dosis for water equivalent accumulated over time t with absorber ab

    Typical setup for an USAXS experiment at the P10 coherence applications beam line at DESY is used
    

    Parameters
    ----------
    t: double
        exposure time in seconds
    ab: int
        number of 25 micron silicon absorbers inserted before the sample


    Returns
    -------
    double
        dosis in kilo gray
    
    '''
    Phi_c = 6e10*(0.729**ab)
    E_c =8540*e  #J/ph photon energy
    d_s =0.0015 #m sample thickness
    A = 0.72# absorption in 1mm of water
    Phi = 1 # volume fraction
    z_sq = 100e-06*100e-06 #m^2 beam area

    rho = 1000 #kg/m^3 sample mass density
    return(np.multiply(t,Phi_c*E_c*A*Phi/(z_sq*d_s*rho))/1000) #return Dose in kGy


In [ ]:
dataSets = pd.read_csv('measurement_set.csv', delimiter=';',header=0,)
print(dataSets)

In [ ]:
plt.style.use('paper_style.mplstyle') # style file for all plots

# Figure 2

In [ ]:
%matplotlib notebook
fig, (ax0, ax1) = plt.subplots(ncols=2,figsize=(7,2.5))
plt.subplots_adjust(left=0.1,
                    bottom=0.15,
                    right=0.84,
                    top=0.93,
                    wspace=0.3,
                    hspace=0.4)
for a in [ax0,ax1]:
    a.set_ylabel(r'$I\mathrm{\:(ph\:pix^{-1}\:s^{-1})}$')
    a.set_xlabel(r'$q\mathrm{\:(nm^{-1})}$')
    a.set_yscale('log')
    a.set_xscale('log')
    
abs_colors = plt.cm.turbo_r(np.linspace(0,1,10))
    
pos0 = ax0.get_position() # get the original position 
ax0a = fig.add_axes([pos0.x0+.6*pos0.x0, pos0.y0*1.8, pos0.width*0.3, pos0.height*0.3])
ax0.text(-0.15,0.98,'a',fontsize=18,transform = ax0.transAxes,fontweight='bold')
ax0.text(0.5,0.75,'$T_{\mathrm{prep}}=63\:\degree$C',fontsize=12,transform = ax0.transAxes)

pos1 = ax1.get_position() # get the original position 
ax1a = fig.add_axes([pos1.x0+.6*pos0.x0, pos1.y0*1.8, pos1.width*0.3, pos1.height*0.3])
ax1.text(-0.15,0.98,'b',fontsize=18,transform = ax1.transAxes,fontweight='bold')
ax1.text(0.5,0.75,'$T_{\mathrm{prep}}=78\:\degree$C',fontsize=12,transform = ax1.transAxes)


ax_col_tw = fig.add_axes([0.91,0.15,0.02,pos0.height])
ax_col_dose = fig.add_axes([0.9099,0.15,0.02,pos0.height])

pos0a=ax0a.get_position()
ax_col_doserate0 = fig.add_axes([pos0a.x0,pos0a.y0+pos0a.height+0.01,pos0a.width,0.02])
pos1a=ax1a.get_position()
ax_col_doserate1 = fig.add_axes([pos1a.x0,pos1a.y0+pos1a.height+0.01,pos1a.width,0.02])

for a in [ax0a,ax1a]:
    a.set_xscale('log')
    a.set_xlabel(r'$\mathcal{D}\:(\mathrm{kGy})$',fontsize=8,labelpad=-2)
ax0a.set_ylabel(r'$I/I_0$',fontsize=8)
ax1a.set_ylabel(r'$I/I_0$',fontsize=8,labelpad=-5)
    
#ax1a_set_xticks(1,10,100)
ax1a.set_ylim([0.94,1.007])
ax0a.set_xlim([0.3,250])
ax1a.set_xlim([0.9,350])
ax0a.set_ylim([0.78,1.12])
ax0a.axhline(1,color='black',zorder=-1)
ax1a.axhline(1,color='black',zorder=-1)


ax0.set_ylim([0.00008,20])
ax1.set_ylim([0.00008,20])
measurement_no = 4
with h5py.File('hdf5Results/' + dataSets['measurement'][measurement_no]+'.hdf5','r') as f:
           
    ser = 16
    cut_end=-300
    I = f['/scan_{:.0f}/SAXS/profiles/I'.format(ser)][:,:cut_end]
    q = f['/scan_{:.0f}/SAXS/profiles/q'.format(ser)][:cut_end]
    tw = f['/scan_{:.0f}/SAXS/profiles/t_wait_s'.format(ser)][()]
    colors= plt.cm.RdYlBu_r(0.5*(np.log(np.linspace(0.01,1,I.shape[0]))/np.log(10)+2)) ############################   

    for i in range(0,I.shape[0],4):
        ax0.plot(q,I[i,:],color=colors[i],linewidth=1,path_effects=[pe.Stroke(linewidth=1.1, foreground='black',alpha=1), pe.Normal()])
        
        
    #create colorbar
    cmap = plt.cm.RdYlBu_r
    norm = mpl.colors.LogNorm(vmin=tw[1], vmax=tw[-1])
    cb1 = mpl.colorbar.ColorbarBase(ax_col_tw, cmap=cmap,
                                    norm=norm,
                                    orientation='vertical')
    cb1.set_label('$t_{\mathrm{wait}}\:(\mathrm{s})}$')
    norm2 = mpl.colors.LogNorm(vmin=Dosis(tw[1],0), vmax=Dosis(tw[-1],0))
    cb2 = mpl.colorbar.ColorbarBase(ax_col_dose, cmap=cmap,
                                    norm=norm2,
                                    orientation='vertical')
    cb2.set_label('$\mathcal{D}\:(\mathrm{kGy})$')
    
for im,meas_code in enumerate([25]):
    colors = plt.cm.turbo_r(np.linspace(0,1,10))
    with h5py.File(f'hdf5Results/m0{meas_code}_IntensityAnalysis.h5py','r') as f:
        for ia, absb in enumerate([12,8,6,4,2,1,0]):#18,16,12,
            
            x=f[f'/abs_{absb}/Dose_wait'][()]
            y=f[f'/abs_{absb}/I_mean_xpcs'][()]
            dy = f[f'/abs_{absb}/dI_mean_xpcs'][()]
            y0=f[f'/abs_{absb}/I_mean_xpcs'][1]
            dy0 = f[f'/abs_{absb}/dI_mean_xpcs'][1]


            x_logave, y_logave, dy_logave = log_ave_errors(x,y,dy,30)

            y_y0_logave = np.divide(y_logave,y0)
            dy_y0_logave = np.sqrt(np.add(np.power(np.divide(dy_logave,y0),2),np.power(np.divide(dy0*y_logave,np.power(y0,2)),2)))

#                 print(y_y0_logave[3])
#                 print(dy_y0_logave[3])

            ind_thres = np.where(y_y0_logave>1.01)[0]
            if len(ind_thres>0):
                ax0a.axvline(x=x_logave[ind_thres[0]], ymin=0, ymax=0.2, color=colors[ia+3])
            ax0a.errorbar(x=x_logave,y=y_y0_logave,
                         #yerr=dy_y0_logave,
                         color=colors[ia+3],marker='o',
                         mec='gray',mew=0.3,linestyle='',
                         markersize=3,zorder=absb,alpha=1,elinewidth=0.5)
    

        
measurement_no = 10
with h5py.File('hdf5Results/' + dataSets['measurement'][measurement_no]+'.hdf5','r') as f:
    ser = 15
    cut_end=-300
    I = f['/scan_{:.0f}/SAXS/profiles/I'.format(ser)][:,:cut_end]
    q = f['/scan_{:.0f}/SAXS/profiles/q'.format(ser)][:cut_end]
    colors= plt.cm.RdYlBu_r(0.5*(np.log(np.linspace(0.01,1,I.shape[0]))/np.log(10)+2))############
    for i in range(0,I.shape[0],4):
        ax1.plot(q,I[i,:],color=colors[i],linewidth=1,path_effects=[pe.Stroke(linewidth=1.1, foreground='black',alpha=1), pe.Normal()])
        
for im,meas_code in enumerate([50]):
    colors = plt.cm.turbo_r(np.linspace(0,1,10))
    with h5py.File(f'hdf5Results/m0{meas_code}_IntensityAnalysis_300.h5py','r') as f:
        for ia, (absb,ser) in enumerate(zip([8,6,4,2,1,0],[5,7,9,11,13,15])):#18,16,12,
            x=f[f'/abs_{absb}/Dose_wait'][()]
            y=f[f'/abs_{absb}/I_mean_xpcs'][()]
            dy = f[f'/abs_{absb}/dI_mean_xpcs'][()]
            y0=f[f'/abs_{absb}/I_mean_xpcs'][1]
            dy0 = f[f'/abs_{absb}/dI_mean_xpcs'][1]


            x_logave, y_logave, dy_logave = log_ave_errors(x,y,dy,30)

            y_y0_logave = np.divide(y_logave,y0)
            dy_y0_logave = np.sqrt(np.add(np.power(np.divide(dy_logave,y0),2),np.power(np.divide(dy0*y_logave,np.power(y0,2)),2)))

#                 print(y_y0_logave[3])
#                 print(dy_y0_logave[3])

            ind_thres = np.where(y_y0_logave<=0.99)[0]
            if len(ind_thres>0):
                ax1a.axvline(x=x_logave[ind_thres[0]], ymin=0, ymax=0.2, color=colors[ia+4])
            yerr = np.std(y_y0_logave[:3])
            ax1a.errorbar(x=x_logave,y=y_y0_logave,
                         #yerr=dy_y0_logave,
                         color=colors[ia+4],marker='o',
                         mec='gray',mew=0.3,linestyle='',
                         markersize=3,zorder=absb,alpha=1,elinewidth=0.5)
            
            
ax_col_tw.yaxis.set_ticks_position("left")
ax_col_tw.yaxis.set_label_position("left")

ax_col_dose.yaxis.set_ticks_position("right")
ax_col_dose.yaxis.set_label_position("right")

#colorbar for inset plot
mpl.rcParams['axes.labelsize'] = 8
mpl.rcParams['ytick.labelsize'] = 8
cmap = ListedColormap([abs_colors[3],abs_colors[4],abs_colors[5],abs_colors[6],abs_colors[7],abs_colors[8],abs_colors[9]])
norm = mpl.colors.Normalize(vmin=0,vmax=7)
cb0a = mpl.colorbar.ColorbarBase(ax_col_doserate0, cmap=cmap,
                                    norm=norm,
                                    orientation='horizontal')
ax_col_doserate0.xaxis.set_ticks([0.5,3.5,6.5])
ax_col_doserate0.xaxis.set_ticklabels([0.09,1,4])
cb0a.set_label('$\mathscr{D}\:\mathrm{(kGy\:s^{-1})}$',fontsize=8,labelpad=1)
ax_col_doserate0.xaxis.set_ticks_position("top")
ax_col_doserate0.xaxis.set_label_position("top")
ax_col_doserate0.tick_params(axis='x', which='major', pad=-1.5)

cmap = ListedColormap([abs_colors[5],abs_colors[6],abs_colors[7],abs_colors[8],abs_colors[9]])
norm = mpl.colors.Normalize(vmin=0,vmax=5)
cb0a = mpl.colorbar.ColorbarBase(ax_col_doserate1, cmap=cmap,
                                    norm=norm,
                                    orientation='horizontal')
ax_col_doserate1.xaxis.set_ticks([0.5,2.5,4.5])
ax_col_doserate1.xaxis.set_ticklabels([0.6,2,4])
cb0a.set_label('$\mathscr{D}\:\mathrm{(kGy\:s^{-1})}$',fontsize=8,labelpad=1)
ax_col_doserate1.xaxis.set_ticks_position("top")
ax_col_doserate1.xaxis.set_label_position("top")

plt.savefig('Figure2.pdf',dpi=1200,format='pdf')

# Figure 7

In [ ]:
fig, (ax0, ax1) = plt.subplots(ncols=2,figsize=(3.47,1.25))
plt.subplots_adjust(left=0.11,
                    bottom=0.25,
                    right=0.7,
                    top=0.85,
                    wspace=0.43,
                    hspace=0.4)
for a in [ax0,ax1]:
    a.set_ylabel(r'$I/I_0$',labelpad=-10)
    a.set_xlabel(r'$\Gamma(\Phi,\mathcal{D})\cdot t$')
    a.set_xscale('log')

ax0.text(-0.2,0.98,'a',fontsize=12,transform = ax0.transAxes,fontweight='bold')
ax0.text(0.05,0.35,'$T_{\mathrm{prep}}=63\:\degree$C',fontsize=6,transform = ax0.transAxes,zorder=30)
ax1.text(-0.2,0.98,'b',fontsize=12,transform = ax1.transAxes,fontweight='bold')
ax1.text(0.05,0.35,'$T_{\mathrm{prep}}=78\:\degree$C',fontsize=6,transform = ax1.transAxes,zorder=30)

ax1.set_yticks([0.94,1])
ax1.set_yticklabels([0.94,1])
ax1.set_ylim([0.935,1.01])
ax0.set_yticks([0.85,1])
ax0.set_yticklabels([0.85,1])

for im,meas_code in enumerate([25]):
    colors = plt.cm.turbo_r(np.linspace(1,0,10))
    with h5py.File(f'hdf5Results/m0{meas_code}_IntensityAnalysis.h5py','r') as f:
        with h5py.File(f'hdf5Results/m025_eggwhite_63C_bd.hdf5','r') as g:
            N_maxes = []
            for ia, (absb,ser) in enumerate(zip([12,8,6,4,2,1,0],[4,6,7,9,11,13,15])):#18,16,12,
                gamma= g[f'/scan_{ser}/q_0/horizontal_cuts/1_exp_fit/fit_parameter_results'][:,1]
                tcut = g[f'/scan_{ser}/q_0/horizontal_cuts/1_exp_fit/t_cut'][()]
                x=f[f'/abs_{absb}/t_wait'][()]
                
                gamma_for_x = np.interp(x,tcut,gamma)
                x_prime = np.multiply(x,gamma_for_x)
                y=f[f'/abs_{absb}/I_mean_xpcs'][()]
                dy = f[f'/abs_{absb}/dI_mean_xpcs'][()]
                y0=f[f'/abs_{absb}/I_mean_xpcs'][1]
                dy0 = f[f'/abs_{absb}/dI_mean_xpcs'][1]
                
                t_logave, _ = log_ave(x,y,15)
                x_logave, y_logave, dy_logave = log_ave_errors(x_prime,y,dy,15)
                
                
                y_y0_logave = np.divide(y_logave,y0)
                dy_y0_logave = np.sqrt(np.add(np.power(np.divide(dy_logave,y0),2),np.power(np.divide(dy0*y_logave,np.power(y0,2)),2)))
                
                
                ind_thres = np.where(y_y0_logave>1.01)[0]
                if len(ind_thres>0):
                    N_maxes.append(x_logave[ind_thres[0]])
                    ax0.axvline(x=x_logave[ind_thres[0]], ymin=0, ymax=0.2, color=colors[ia+3])
                ax0.errorbar(x=x_logave,y=y_y0_logave,
                             color=colors[ia+3],marker='o',
                             mec='gray',mew=0.3,linestyle='',
                             markersize=4,zorder=absb,alpha=1,elinewidth=0.5)


for im,meas_code in enumerate([50]):
    colors = plt.cm.turbo_r(np.linspace(1,0,10))
    with h5py.File(f'hdf5Results/m0{meas_code}_IntensityAnalysis_300.h5py','r') as f:
        with h5py.File(f'hdf5Results/m0{meas_code}_eggwhite_78C_bd.hdf5','r') as g:
            ser_list = [5,9,11,13,15]
            abs_list = [8,4,2,1,0]
            q_ind = 5
            nexp = [1,1,2,2,2]
            N_maxes = []
            for ia, (absb, ser, nex) in enumerate(zip(abs_list,ser_list,nexp)):#18,16,12,
                if nex ==1:

                    gamma= g[f'/scan_{ser}/q_{q_ind}/horizontal_cuts/1_exp_fit/fit_parameter_results'][:,1]
                    tcut = g[f'/scan_{ser}/q_{q_ind}/horizontal_cuts/1_exp_fit/t_cut'][()]
                elif nex ==2:
                    Gamma1= g[f'/scan_{ser}/q_{q_ind}/horizontal_cuts/2_exp_fit/fit_parameter_results'][:,2]
                    Gamma2= g[f'/scan_{ser}/q_{q_ind}/horizontal_cuts/2_exp_fit/fit_parameter_results'][:,4]
                    
                    Gammas = np.stack([Gamma1,Gamma2])
                    Gsort = np.sort(Gammas,axis=0)
                    
                    gamma = Gsort[0,:]
                    
                    tcut = g[f'/scan_{ser}/q_{q_ind}/horizontal_cuts/2_exp_fit/t_cut'][()]
                x=f[f'/abs_{absb}/t_wait'][()]
                
                gamma_for_x = np.interp(x,tcut,gamma)
                y=f[f'/abs_{absb}/I_mean_xpcs'][()]
                dy = f[f'/abs_{absb}/dI_mean_xpcs'][()]
                y0=f[f'/abs_{absb}/I_mean_xpcs'][1]
                dy0 = f[f'/abs_{absb}/dI_mean_xpcs'][1]
                
                x_prime = np.multiply(x,gamma_for_x)
                
                t_logave, _ = log_ave(x,y,15)
                x_logave, y_logave, dy_logave = log_ave_errors(x_prime,y,dy,25)
                

                y_y0_logave = np.divide(y_logave,y0)
                dy_y0_logave = np.sqrt(np.add(np.power(np.divide(dy_logave,y0),2),np.power(np.divide(dy0*y_logave,np.power(y0,2)),2)))
                
                
                ind_thres = np.where(y_y0_logave<0.99)[0]
                if len(ind_thres>0):
                    N_maxes.append(x_logave[ind_thres[0]])
                    ax1.axvline(x=x_logave[ind_thres[0]], ymin=0, ymax=0.2, color=colors[ia+5])
                ax1.errorbar(x=x_logave,y=y_y0_logave,
                             color=colors[ia+5],marker='o',
                             mec='gray',mew=0.3,linestyle='',
                             markersize=4,zorder=absb,alpha=1,elinewidth=0.5)

ax_col_flux = fig.add_axes([0.83,0.15,0.02,pos0.height])
ax_col_doserate = ax_col_flux.twinx()

cmap = ListedColormap([colors[i] for i in range(3,10)])
norm = mpl.colors.Normalize(vmin=0,vmax=7)
cb = mpl.colorbar.ColorbarBase(ax_col_doserate, cmap=cmap,norm=norm,orientation='vertical')
cb.set_ticks(np.arange(0.5,7.5,1))
cb.set_ticklabels([0.09,0.3,0.6,1,2,3,4])
cb.set_label('$\mathscr{D}\:\mathrm{(kGy\:s^{-1})}$',fontsize=8,labelpad=0)

ax_col_flux.set_ylim([0,7])
ax_col_flux.set_yticks(np.arange(0.5,7.5,1))
ax_col_flux.set_yticklabels([r'0.1',r'0.5',r'0.9',r'2',r'3',r'4',r'6'],fontsize=8)

ax_col_flux.set_ylabel(r'$\Phi$ ($\mathrm{ph}\:\mathrm{s}^{-1}\:\mathrm{nm}^{-2}$)',fontsize=8)

plt.savefig('Figure7.pdf',dpi=1200,format='pdf')

# Figure S2

In [ ]:
meas_list = [0,1,2,3,4,5,6,7,8,9,10,11,12]
scan_list = [13,13,13,14,2,2,2,2,2,2,2,2,2,2]
temp = np.array([50,53,57,60,63,65, 68, 70, 73, 75, 78,80,85])
cut_end = -300

fig, ax = plt.subplots()
colors = np.concatenate((plt.cm.binary(np.linspace(0,1,4)),plt.cm.seismic(np.linspace(0.15,1,9))),axis=0)


for im, (t,meas_id, scan) in enumerate(zip(temp,meas_list,scan_list)):
    with h5py.File('hdf5Results/' + dataSets['measurement'][meas_id]+'.hdf5','r') as f:
        I = f[f'/scan_{scan}/SAXS/profiles/I'][0,:cut_end]
        q = f[f'/scan_{scan}/SAXS/profiles/q'][:cut_end]
        ax.plot(q,I,color=colors[im],linewidth=2,path_effects=[pe.Stroke(linewidth=3, foreground='black',alpha=0.5), pe.Normal()],)
        
ax.set_xscale('log')
ax.set_yscale('log')

ax.set_ylabel('I (ph $\mathrm{pixel}^{-1}\:\mathrm{s}^{-1}$)')
ax.set_xlabel('q($\mathrm{nm^{-1}}$)')

ax.legend(temp,title='$\mathrm{T_{prep}}(\degree C)$',ncol=2)
plt.tight_layout()

plt.savefig('FigureS2.pdf',dpi=1200,format='pdf')

# Figure S3

In [ ]:
fig, axes = plt.subplots(nrows=3,ncols=5,
                         sharex=True,sharey=True,
                         squeeze=True,figsize=(7,7.5))
ax_all = plt.axes([0,0,1,1])
ax_all.axis('off')
plt.subplots_adjust(right=0.78,wspace=0,hspace=0.22,left=0.1,bottom=0.1,top=0.95)

colors = plt.cm.turbo_r(np.linspace(1,0,10))
ax_col_flux = fig.add_axes([0.88,0.1,0.02,0.85])
ax_col_doserate = ax_col_flux.twinx()

cmap = ListedColormap([colors[i] for i in range(10)])
norm = mpl.colors.Normalize(vmin=0,vmax=10)
cb = mpl.colorbar.ColorbarBase(ax_col_doserate, cmap=cmap,norm=norm,orientation='vertical')
cb.set_ticks(np.arange(0.5,10.5,1))
cb.set_ticklabels([0.002,0.014,0.03,0.09,0.33,0.61,1.13,2.21,2.89,3.95])
cb.set_label('$\mathscr{D}\:\mathrm{(kGy\:s^{-1})}$',fontsize=12,labelpad=0)

ax_col_flux.set_ylim([0,10])
ax_col_flux.set_yticks(np.arange(0.5,10.5,1))
ax_col_flux.set_yticklabels([r'3$\times 10^{-3}$',r'2$\times 10^{-2}$',r'4$\times 10^{-2}$',r'1$\times 10^{-1}$',
                             r'5$\times 10^{-1}$',r'9$\times 10^{-1}$',r'2$\times 10^{0}$',r'3$\times 10^{0}$',
                             r'4$\times 10^{0}$',r'6$\times 10^{0}$'])

ax_col_flux.set_ylabel(r'$\Phi$ ($\mathrm{ph}\:\mathrm{s}^{-1}\:\mathrm{nm}^{-2}$)')

color_dict = {'24':colors[0],'18':colors[1],'16':colors[2],'12':colors[3],'8':colors[4],'6':colors[5],
              '4':colors[6],'2':colors[7],'1':colors[8],'0':colors[9]}
cut_end = -300
ax = axes.flatten()

#50C
measurement_no = 0
with h5py.File('hdf5Results/' + dataSets['measurement'][measurement_no]+'.hdf5','r') as f:
    for iser, (ser, ab) in enumerate(zip([13,2,4,6,8,10,12],[18,8,6,4,2,1,0])):
        divide = np.divide(f['/scan_{:.0f}/SAXS/profiles/I'.format(ser)][-1,:cut_end],1)
        ax[0].plot(f['/scan_{:.0f}/SAXS/profiles/q'.format(ser)][:cut_end], 
                divide,
                color=color_dict[str(ab)])

#53C
measurement_no = 1
with h5py.File('hdf5Results/' + dataSets['measurement'][measurement_no]+'.hdf5','r') as f:
    for iser, (ser, ab) in enumerate(zip([13,2,4,6,8,10,12],[18,8,6,4,2,1,0])):
        divide = np.divide(f['/scan_{:.0f}/SAXS/profiles/I'.format(ser)][-1,:cut_end],1)
        ax[1].plot(f['/scan_{:.0f}/SAXS/profiles/q'.format(ser)][:cut_end], 
                divide,
                color=color_dict[str(ab)])

#57C
measurement_no = 2
with h5py.File('hdf5Results/' + dataSets['measurement'][measurement_no]+'.hdf5','r') as f:
    for iser, (ser, ab) in enumerate(zip([13,2,4,6,8,10,12],[18,8,6,4,2,1,0])):
        divide = np.divide(f['/scan_{:.0f}/SAXS/profiles/I'.format(ser)][-1,:cut_end],1)
        ax[2].plot(f['/scan_{:.0f}/SAXS/profiles/q'.format(ser)][:cut_end], 
                divide,
                color=color_dict[str(ab)])
print(60)
#60C
measurement_no = 3
with h5py.File('hdf5Results/' + dataSets['measurement'][measurement_no]+'.hdf5','r') as f:
    for iser, (ser, ab) in enumerate(zip([14,1,3,5,7,9,11,13],[18,12,8,6,4,2,1,0])):
        divide = np.divide(f['/scan_{:.0f}/SAXS/profiles/I'.format(ser)][-1,:cut_end],1)
        ax[3].plot(f['/scan_{:.0f}/SAXS/profiles/q'.format(ser)][:cut_end], 
                divide,
                color=color_dict[str(ab)])

        #63
measurement_no = 4
with h5py.File('hdf5Results/' + dataSets['measurement'][measurement_no]+'.hdf5','r') as f:
    for iser, (ser, ab) in enumerate(zip([1,2,3,4,6,8,10,12,14,16],[24,18,16,12,8,6,4,2,1,0])):
        divide = np.divide(f['/scan_{:.0f}/SAXS/profiles/I'.format(ser)][-1,:cut_end],1)
        ax[5].plot(f['/scan_{:.0f}/SAXS/profiles/q'.format(ser)][:cut_end], 
                divide,
                color=color_dict[str(ab)])

#65
measurement_no = 5
with h5py.File('hdf5Results/' + dataSets['measurement'][measurement_no]+'.hdf5','r') as f:
    for iser, (ser, ab) in enumerate(zip([1,2,3,4,6,8,10,12,14,16],[24,18,16,12,8,6,4,2,1,0])):
        divide = np.divide(f['/scan_{:.0f}/SAXS/profiles/I'.format(ser)][-1,:cut_end],1)
        ax[6].plot(f['/scan_{:.0f}/SAXS/profiles/q'.format(ser)][:cut_end], 
                divide,
                color=color_dict[str(ab)])

#68
measurement_no = 6
with h5py.File('hdf5Results/' + dataSets['measurement'][measurement_no]+'.hdf5','r') as f:
    for iser, (ser, ab) in enumerate(zip([1,2,3,4,6,8,10,12,14,16],[24,18,16,12,8,6,4,2,1,0])):
        divide = np.divide(f['/scan_{:.0f}/SAXS/profiles/I'.format(ser)][-1,:cut_end],1)
        ax[7].plot(f['/scan_{:.0f}/SAXS/profiles/q'.format(ser)][:cut_end], 
                divide,
                color=color_dict[str(ab)])
        
        
print(70)      
#70
measurement_no = 7
with h5py.File('hdf5Results/' + dataSets['measurement'][measurement_no]+'.hdf5','r') as f:
    for iser, (ser, ab) in enumerate(zip([1,2,3,4,6,8,10,12,14,16],[24,18,16,12,8,6,4,2,1,0])):
        print(ser)
        divide = np.divide(f['/scan_{:.0f}/SAXS/profiles/I'.format(ser)][-1,:cut_end],1)
        ax[8].plot(f['/scan_{:.0f}/SAXS/profiles/q'.format(ser)][:cut_end], 
                divide,
                color=color_dict[str(ab)])
print(73)        
#73
measurement_no = 8
with h5py.File('hdf5Results/' + dataSets['measurement'][measurement_no]+'.hdf5','r') as f:
    for iser, (ser, ab) in enumerate(zip([1,2,3,4,5,7,9,11,13,15],[24,18,16,12,8,6,4,2,1,0])):
        divide = np.divide(f['/scan_{:.0f}/SAXS/profiles/I'.format(ser)][-1,:cut_end],1)
        ax[10].plot(f['/scan_{:.0f}/SAXS/profiles/q'.format(ser)][:cut_end], 
                divide,
                color=color_dict[str(ab)])
print(75)       
#75
measurement_no = 9
with h5py.File('hdf5Results/' + dataSets['measurement'][measurement_no]+'.hdf5','r') as f:
    for iser, (ser, ab) in enumerate(zip([1,2,3,4,5,7,9,11,13,15],[24,18,16,12,8,6,4,2,1,0])):
        divide = np.divide(f['/scan_{:.0f}/SAXS/profiles/I'.format(ser)][-1,:cut_end],1)
        ax[11].plot(f['/scan_{:.0f}/SAXS/profiles/q'.format(ser)][:cut_end], 
                divide,
                color=color_dict[str(ab)])

#78
measurement_no = 10
with h5py.File('hdf5Results/' + dataSets['measurement'][measurement_no]+'.hdf5','r') as f:
    for iser, (ser, ab) in enumerate(zip([1,2,3,4,5,7,9,11,13,15],[24,18,16,12,8,6,4,2,1,0])):
        divide = np.divide(f['/scan_{:.0f}/SAXS/profiles/I'.format(ser)][-1,:cut_end],1)
        ax[12].plot(f['/scan_{:.0f}/SAXS/profiles/q'.format(ser)][:cut_end], 
                divide,
                color=color_dict[str(ab)])
print(80)      
#80
measurement_no = 11
with h5py.File('hdf5Results/' + dataSets['measurement'][measurement_no]+'.hdf5','r') as f:
    for iser, (ser, ab) in enumerate(zip([1,2,3,4,5,7,9,11,13,15],[24,18,16,12,8,6,4,2,1,0])):
        divide = np.divide(f['/scan_{:.0f}/SAXS/profiles/I'.format(ser)][-1,:cut_end],1)
        ax[13].plot(f['/scan_{:.0f}/SAXS/profiles/q'.format(ser)][:cut_end], 
                divide,
                color=color_dict[str(ab)])
        
print(85)
measurement_no = 12
with h5py.File('hdf5Results/' + dataSets['measurement'][measurement_no]+'.hdf5','r') as f:
    for iser, (ser, ab) in enumerate(zip([1,2,3,4,6,8,10],[24,16,12,8,4,2,0])):
        divide = np.divide(f['/scan_{:.0f}/SAXS/profiles/I'.format(ser)][-1,:cut_end],1)
        ax[14].plot(f['/scan_{:.0f}/SAXS/profiles/q'.format(ser)][:cut_end], 
                divide,
                color=color_dict[str(ab)])

titles = ['50$\degree$C','53$\degree$C','57$\degree$C','60$\degree$C','63$\degree$C',
          '65$\degree$C','68$\degree$C','70$\degree$C','73$\degree$C','75$\degree$C','78$\degree$C','80$\degree$C','85$\degree$C']
for ia,a in enumerate(ax[[0,1,2,3,5,6,7,8,10,11,12,13,14]]):
    a.set_xscale('log')
    #a.axhline(1)
    a.set_yscale('log')
    a.text(0.5,0.9,titles[ia],fontsize=12,transform = a.transAxes,color='black')
    #a.set_title(titles[ia])

ax[2].set_title('samples with small fraction of aggregated proteins',weight='normal')
ax[7].set_title('soft gel networks',weight='normal')
ax[12].set_title('strong gel networks',weight='normal')

    
ax[4].plot(.1,1)
ax[9].plot(.1,1)
ax[4].axis('off')
ax[9].axis('off')
    
#ax[0].set_ylim([0,3])    
ax[0].set_ylabel(r'$\mathrm{I_{end}\:(ph\:pix^{-1}\:s^{-1})}$',fontsize=12)
ax[5].set_ylabel(r'$\mathrm{I_{end}\:(ph\:pix^{-1}\:s^{-1})}$',fontsize=12)
ax[10].set_ylabel(r'$\mathrm{I_{end}\:(ph\:pix^{-1}\:s^{-1})}$',fontsize=12)

ax[10].set_xlabel('q($nm^{-1}$)')
ax[11].set_xlabel('q($nm^{-1}$)')
ax[12].set_xlabel('q($nm^{-1}$)')
ax[13].set_xlabel('q($nm^{-1}$)')
ax[14].set_xlabel('q($nm^{-1}$)')

plt.savefig('FigureS3.pdf',format='pdf',dpi=fig.dpi)

# Figure S4

In [ ]:
fig, axes = plt.subplots(nrows=3,ncols=5,
                         sharex=True,sharey=True,
                         squeeze=True,figsize=(7,7))
plt.subplots_adjust(right=0.78,wspace=0,hspace=0.22,left=0.1,bottom=0.1,top=0.95)

ax_col_tw = fig.add_axes([0.86,0.1,0.02,0.85])
ax_col_dose = fig.add_axes([0.8599,0.1,0.01999,0.85])

#create colorbar
cmap = mpl.cm.RdYlBu_r
norm = mpl.colors.LogNorm(vmin=tw[1], vmax=tw[-1])
cb1 = mpl.colorbar.ColorbarBase(ax_col_tw, cmap=cmap,
                                norm=norm,
                                orientation='vertical')
cb1.set_label('$t_{\mathrm{wait}}\:(\mathrm{s})}$')
norm2 = mpl.colors.LogNorm(vmin=Dosis(tw[1],0), vmax=Dosis(tw[-1],0))
cb2 = mpl.colorbar.ColorbarBase(ax_col_dose, cmap=cmap,
                                norm=norm2,
                                orientation='vertical')
cb2.set_label('$\mathcal{D}\:(\mathrm{kGy})$')
    

            
ax_col_tw.yaxis.set_ticks_position("left")
ax_col_tw.yaxis.set_label_position("left")

ax_col_dose.yaxis.set_ticks_position("right")
ax_col_dose.yaxis.set_label_position("right")

color_dict = {'24':colors[0],'18':colors[1],'16':colors[2],'12':colors[3],'8':colors[4],'6':colors[5],
              '4':colors[6],'2':colors[7],'1':colors[8],'0':colors[9]}
cut_end = -300
ax_all = axes.flatten()
ax = ax_all[[0,1,2,3,5,6,7,8,10,11,12,13,14]]


for jj, (measurement_no,ser,a) in enumerate(zip([0,1,2,3,4,5,6,7,8,9,10,11,12],[12,12,12,13,16,16,16,16,15,15,15,15,10],ax)):    

    with h5py.File('hdf5Results/' + dataSets['measurement'][measurement_no]+'.hdf5','r') as f:
        
           
        cut_end=-300
        I = f['/scan_{:.0f}/SAXS/profiles/I'.format(ser)][:,:cut_end]
        q = f['/scan_{:.0f}/SAXS/profiles/q'.format(ser)][:cut_end]
        tw = f['/scan_{:.0f}/SAXS/profiles/t_wait_s'.format(ser)][()]
        colors= plt.cm.RdYlBu_r(0.5*(np.log(np.linspace(0.01,1,I.shape[0]))/np.log(10)+2)) ############################   

        for i in range(I.shape[0]):
            a.plot(q,I[i,:],color=colors[i],linewidth=1)



titles = ['50$\degree$C','53$\degree$C','57$\degree$C','60$\degree$C','63$\degree$C',
          '65$\degree$C','68$\degree$C','70$\degree$C','73$\degree$C','75$\degree$C','78$\degree$C','80$\degree$C','85$\degree$C']
for ia,a in enumerate(ax):
    a.set_xscale('log')
    #a.axhline(1)
    a.set_yscale('log')
    a.text(0.5,0.9,titles[ia],fontsize=12,transform = a.transAxes,color='black')
    #a.set_title(titles[ia])

ax[2].set_title('samples with small fraction of aggregated proteins',weight='normal')
ax[6].set_title('soft gel networks',weight='normal')
ax[10].set_title('strong gel networks',weight='normal')

    
ax_all[4].plot(.1,1)
ax_all[9].plot(.1,1)
ax_all[4].axis('off')
ax_all[9].axis('off')
    
#ax[0].set_ylim([0,3])    
ax[0].set_ylabel((r'$I\mathrm{\:(ph\:pix^{-1}\:s^{-1})}$'),fontsize=12)
ax[4].set_ylabel((r'$I\mathrm{\:(ph\:pix^{-1}\:s^{-1})}$'),fontsize=12)
ax[8].set_ylabel((r'$I\mathrm{\:(ph\:pix^{-1}\:s^{-1})}$'),fontsize=12)

ax[8].set_xlabel('$q$($\mathrm{nm^{-1}}$)')
ax[9].set_xlabel('$q$($\mathrm{nm^{-1}}$)')
ax[10].set_xlabel('$q$($\mathrm{nm^{-1}}$)')
ax[11].set_xlabel('$q$($\mathrm{nm^{-1}}$)')
ax[12].set_xlabel('$q$($\mathrm{nm^{-1}}$)')

plt.savefig('FigureS4.pdf',dpi=1200,format='pdf')

# Figure S5

In [ ]:
fig, axes = plt.subplots(nrows=2,ncols=5,
                         sharex=True,sharey='row',
                         squeeze=True,figsize=(7,4))
plt.subplots_adjust(right=0.78,wspace=0,hspace=0.3,left=0.1,bottom=0.1,top=0.95)


color_dict = {'24':colors[0],'18':colors[1],'16':colors[2],'12':colors[3],'8':colors[4],'6':colors[5],
              '4':colors[6],'2':colors[7],'1':colors[8],'0':colors[9]}

ax=axes.flatten()[[0,1,2,3,5,6,7,8,9]]

for ii,(a,degrees) in enumerate(zip(ax,[63,65,68,70,73,75,78,80,85])):

    a.set_xscale('log')
    a.text(0.1,0.3,f'{degrees}'+r'$\:\degree$C',fontsize=12,transform = a.transAxes)
    a.grid('on',zorder=-10)
    
colors = plt.cm.turbo_r(np.linspace(1,0,10))
absorber_index = {'24':0,'18':1,'16':2,'12':3,'8':4,'6':5,'4':6,'2':7,'1':8,'0':9}
flux_dict={'24':3.36e7,'18':2.18e8,'16':4.08e8,'12':1.42e9,'8':4.95e9,'6':9.23e9,'4':1.72e10,'2':3.21e10,'1':4.39e10,'0':6e10,}
doserate_dict = {'24':3.36e7,'18':2.18e8,'16':0.02,'12':0.09,'8':0.32,'6':0.60,'4':1.13,'2':2.11,'1':2.89,'0':3.9}
for im,(meas_code,a,degrees) in enumerate(zip([25,26,28,29,51,60,50,46,59],ax,[63,65,68,70,73,75,78,80,85])):
    print(meas_code)
    with h5py.File(f'hdf5Results/m0{meas_code}_IntensityAnalysis.h5py','r') as f:
       
        abs_list = [8,6,4,2,1,0]
        if degrees == 85:
            abs_list = [8,4,2,0]
        for ia, absb in enumerate(abs_list):#18,16,12,
            
            x=f[f'/abs_{absb}/Dose_wait'][()]
            y=f[f'/abs_{absb}/I_mean_xpcs'][()]
            dy = f[f'/abs_{absb}/dI_mean_xpcs'][()]
            y0=f[f'/abs_{absb}/I_mean_xpcs'][1]
            dy0 = f[f'/abs_{absb}/dI_mean_xpcs'][1]


            x_logave, y_logave, dy_logave = log_ave_errors(x,y,dy,30)

            y_y0_logave = np.divide(y_logave,y0)
            dy_y0_logave = np.sqrt(np.add(np.power(np.divide(dy_logave,y0),2),np.power(np.divide(dy0*y_logave,np.power(y0,2)),2)))


            if degrees<70:
                ind_thres = np.where(y_y0_logave>1.01)[0]
            else:
                ind_thres = np.where(y_y0_logave<0.99)[0]
            if len(ind_thres>0):
                a.axvline(x=x_logave[ind_thres[0]], ymin=0, ymax=0.2, color=colors[ia+3])
            a.errorbar(x=x_logave,y=y_y0_logave,
                         #yerr=dy_y0_logave,
                         color=colors[absorber_index[str(absb)]],marker='o',
                         mec='gray',mew=0.3,linestyle='',
                         markersize=5,zorder=absb+5,alpha=1,elinewidth=0.5)
    


            
ax_col_tw = fig.add_axes([0.91,0.3,0.02,0.4])
       
abs_colors=colors
cmap = ListedColormap([abs_colors[4],abs_colors[5],
                       abs_colors[6],abs_colors[7],abs_colors[8],abs_colors[9]])
norm = mpl.colors.Normalize(vmin=3,vmax=9)
cb0a = mpl.colorbar.ColorbarBase(ax_col_tw, cmap=cmap,
                                    norm=norm,
                                    orientation='vertical')
ax_col_tw.yaxis.set_ticks([3.5,4.5,5.5,6.5,7.5,8.5])
ax_col_tw.yaxis.set_ticklabels([0.3,0.6,1,2,3,4])
cb0a.set_label('$\mathscr{D}\:\mathrm{(kGy\:s^{-1})}$',fontsize=8,labelpad=1)


ax[0].set_ylabel(r'$I/I_0$')
ax[4].set_ylabel(r'$I/I_0$')
ax[4].set_xlabel(r'$\mathcal{D}\:(\mathrm{kGy})$')
ax[5].set_xlabel(r'$\mathcal{D}\:(\mathrm{kGy})$')
ax[6].set_xlabel(r'$\mathcal{D}\:(\mathrm{kGy})$')
ax[7].set_xlabel(r'$\mathcal{D}\:(\mathrm{kGy})$')
ax[8].set_xlabel(r'$\mathcal{D}\:(\mathrm{kGy})$')
    
axes[0,4].plot(.1,1)
axes[0,4].axis('off')
    
axes[0,2].set_title('soft gel networks',weight='normal')
axes[1,2].set_title('strong gel networks',weight='normal')

plt.savefig('FigureS5.pdf',format='pdf',dpi=fig.dpi)

# Figure S6

In [ ]:
fig, axes = plt.subplots(nrows=2,ncols=5,
                         sharey='row',
                         squeeze=True,figsize=(7,4))
plt.subplots_adjust(right=0.78,wspace=0,hspace=0.35,left=0.1,bottom=0.1,top=0.93)

ax=axes.flatten()[[0,1,2,3,5,6,7,8,9]]

for ii,(a,degrees) in enumerate(zip(ax,[63,65,68,70,73,75,78,80,85])):
    a.set_xscale('log')
    a.text(0.1,0.3,f'{degrees}'+'$\:\degree$C',fontsize=12,transform = a.transAxes)
    a.grid('on')
    
colors = plt.cm.turbo_r(np.linspace(1,0,10))
absorber_index = {'24':0,'18':1,'16':2,'12':3,'8':4,'6':5,'4':6,'2':7,'1':8,'0':9}
flux_dict={'24':3.36e7,'18':2.18e8,'16':4.08e8,'12':1.42e9,'8':4.95e9,'6':9.23e9,'4':1.72e10,'2':3.21e10,'1':4.39e10,'0':6e10,}
doserate_dict = {'24':3.36e7,'18':2.18e8,'16':0.02,'12':0.09,'8':0.32,'6':0.60,'4':1.13,'2':2.11,'1':2.89,'0':3.9}
for im,(meas_code,a,degrees) in enumerate(zip([25,26,28,29,51,60,50,46,59],ax,[63,65,68,70,73,75,78,80,85])):
    with h5py.File(f'hdf5Results/m0{meas_code}_IntensityAnalysis.h5py','r') as f:
        with h5py.File(f'hdf5Results/m0{meas_code}_eggwhite_{degrees}C_bd.hdf5','r') as g:
            
            if degrees in [63,65,68,70]:
                ser_list = [6,7,9,11,13,15]
                abs_list = [8,6,4,2,1,0]
                q_ind = 0
                nexp = [1,1,1,1,1,1]
            elif degrees in [73,80]:
                ser_list = [5,7,9,11,13,15]
                abs_list = [8,6,4,2,1,0]
                nexp = [1,1,1,1,1,1]
                q_ind = 5
            elif degrees in [75]:
                ser_list = [5,7,9,11,13,15]
                abs_list = [8,6,4,2,1,0]
                q_ind = 5
                nexp = [1,1,2,2,2,2]
            elif degrees in [78]:
                ser_list = [5,9,11,13,15]
                abs_list = [8,4,2,1,0]
                q_ind = 5
                nexp = [1,1,2,2,2]
            elif degrees in [85]:
                ser_list = [4,6,8,10]
                abs_list = [8,4,2,0]
                nexp = [1,1,1,1]
                q_ind = 5
                
                
            for ia, (absb, ser, nex) in enumerate(zip(abs_list,ser_list,nexp)):#18,16,12,
                if nex ==1:

                    gamma= g[f'/scan_{ser}/q_{q_ind}/horizontal_cuts/1_exp_fit/fit_parameter_results'][:,1]
                    tcut = g[f'/scan_{ser}/q_{q_ind}/horizontal_cuts/1_exp_fit/t_cut'][()]
                elif nex ==2:
                    Gamma1= g[f'/scan_{ser}/q_{q_ind}/horizontal_cuts/2_exp_fit/fit_parameter_results'][:,2]
                    Gamma2= g[f'/scan_{ser}/q_{q_ind}/horizontal_cuts/2_exp_fit/fit_parameter_results'][:,4]
                    
                    Gammas = np.stack([Gamma1,Gamma2])
                    Gsort = np.sort(Gammas,axis=0)
                    
                    gamma = Gsort[0,:]
                    
                    tcut = g[f'/scan_{ser}/q_{q_ind}/horizontal_cuts/2_exp_fit/t_cut'][()]
                x=f[f'/abs_{absb}/t_wait'][()]
                
                gamma_for_x = np.interp(x,tcut,gamma)
                y=f[f'/abs_{absb}/I_mean_xpcs'][()]
                dy = f[f'/abs_{absb}/dI_mean_xpcs'][()]
                y0=f[f'/abs_{absb}/I_mean_xpcs'][1]
                dy0 = f[f'/abs_{absb}/dI_mean_xpcs'][1]
                
                x_prime = np.multiply(x,gamma_for_x)
                
                t_logave, _ = log_ave(x,y,15)
                x_logave, y_logave, dy_logave = log_ave_errors(x_prime,y,dy,25)
                
                y_y0_logave = np.divide(y_logave,y0)
                
                if degrees<71:
                    ind_thres = np.where(y_y0_logave>1.01)[0]
                else:
                    ind_thres = np.where(y_y0_logave<0.99)[0]
                
                if len(ind_thres>0):
                    a.axvline(x=x_logave[ind_thres[0]], ymin=0, ymax=0.2, color=colors[ia+3])
                a.errorbar(x=x_logave,y=y_y0_logave,
                           #xerr=dx,
                            # yerr=dy_y0_logave,
                             color=colors[absorber_index[str(absb)]],marker='o',
                             mec='gray',mew=0.3,linestyle='',
                             markersize=5,zorder=absb+10,alpha=1,elinewidth=0.5)
axes[0,4].plot(1,1)
axes[0,4].axis('off')

ax[0].set_ylabel(r'$I/I_0$')
ax[4].set_ylabel(r'$I/I_0$')
ax[4].set_xlabel(r'$t\cdot \Gamma(\Phi,\mathcal{D})$')
ax[5].set_xlabel(r'$t\cdot \Gamma(\Phi,\mathcal{D})$')
ax[6].set_xlabel(r'$t\cdot \Gamma(\Phi,\mathcal{D})$')
ax[7].set_xlabel(r'$t\cdot \Gamma(\Phi,\mathcal{D})$')
ax[8].set_xlabel(r'$t\cdot \Gamma(\Phi,\mathcal{D})$')
            
ax_col_tw = fig.add_axes([0.91,0.3,0.02,0.4])
       
axes[0,2].set_title('soft gel networks',weight='normal')
axes[1,2].set_title('strong gel networks',weight='normal')

axes[1,1].set_xticks([0.01,0.1,1])
axes[1,1].set_xticklabels(['$10^{-2}$','$10^{-1}$',''])

abs_colors=colors
cmap = ListedColormap([abs_colors[4],abs_colors[5],
                       abs_colors[6],abs_colors[7],abs_colors[8],abs_colors[9]])
norm = mpl.colors.Normalize(vmin=3,vmax=9)
cb0a = mpl.colorbar.ColorbarBase(ax_col_tw, cmap=cmap,
                                    norm=norm,
                                    orientation='vertical')
ax_col_tw.yaxis.set_ticks([3.5,4.5,5.5,6.5,7.5,8.5])
ax_col_tw.yaxis.set_ticklabels([0.3,0.6,1,2,3,4])
cb0a.set_label('$\mathscr{D}\:\mathrm{(kGy\:s^{-1})}$',fontsize=8,labelpad=1)

plt.savefig('FigureS6.pdf',dpi=1200,format='pdf')